# Estudo da associação entre a declaração de raça e acesso à educação superior a partir dos resultados da PNAD Contínua (painel 6)

[Marcelo Pereira](https://github.com/tbmpereira), outubro de 2020

O presente estudo faz parte do artigo entregue para conclusão da disciplina "Sociologia das Desigualdades e da Estratificação", do Programa de Pós-Graduação em Sociologia da Universidade Federal de Minas Gerais, cursada por mim no primeiro semestre de 2020.

In [1]:
import pnadc
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
from scipy.stats import chi2

O [módulo pnadc](https://github.com/ppnasser/pnadc/), criado por Patrick Nasser, importa a rodada trimestral da PNADC e retorna um DataFrame do Pandas. Verificamos que o kernel do Python morre ao importar grandes séries temporais, por problema de memória RAM. Desta maneira, propomos importar cada rodada da PNAD separadamente, filtrar apenas as variáveis de interesse (características gerais do entrevistado e escolaridade), e exportar o 'csv', para posteriormente concatenar os dados.

In [85]:
#gerando tuplas para cada trimestre a ser importado
trimestres = []
for ano in range(2017, 2020):
    for trimestre in range(1, 5):
        trimestres.append((trimestre, ano))

In [ ]:
#criando um laço para importar o dataset de cada trimestre, selecionar as variáveis de interesse e exportar num arquivo csv
for tupla in trimestres:
    df = pnadc.get(tupla[0], tupla[1])
    df = df[['Ano', 'Trimestre', 'UF', 'Capital', 'RM_RIDE', 'UPA', 'Estrato', 'V1008', 'V1014', 'V1016', 'V1022', 'V1023', 'V1027', 'V1028', 
             'V1029', 'posest', 'V2001', 'V2003', 'V2005', 'V2007', 'V2008', 'V20081', 'V20082', 'V2009', 'V2010', 'V3001', 'V3002', 'V3002A', 'V3003', 'V3003A', 'V3007']]
    df = pnadc.tools.identify(df)
    df.to_csv('pnadc_{}-{}.csv'.format(tupla[1], tupla[0]), index=False)

In [ ]:
#concatenar todos os datasets
arquivos = ['pnadc_{}-{}.csv'.format(tupla[1], tupla[0]) for tupla in trimestres]
pnadcs = pd.concat([pd.read_csv(arquivo) for arquivo in arquivos])

In [ ]:
#alterar o tipo das variáveis para salvar memória
for var in pnadcs.columns[:-3]:
    try:
        pnadcs[var] = pnadcs[var].astype('int32')
    except:
        pnadcs[var] = pnadcs[var].astype('float32')

In [ ]:
#exportando o dataset concatenado
pnadcs.to_csv('pnadc_2017-2019.csv', index=False)

In [ ]:
df_chunck = pd.read_csv('pnadc_2017-2019.csv', index_col=False, chunksize=500000)

In [ ]:
def chunk_processing(chunk):
    for var in chunk.columns:
        try:
            chunk[var] = chunk[var].astype('int32')
        except:
            try:
                chunk[var] = chunk[var].astype('float32')
            except:
                continue
    return chunk

chunk_list = []

for chunk in df_chunck:
    chunk_filtered = chunk_processing(chunk)
    chunk_list.append(chunk)

painel6 = pd.concat(chunk_list)

Filtros: Ignorar registros de data de nascimento '9999', Apenas painel 6, descartar variável 'keyDorm'

In [ ]:
painel6.drop('keyDom', axis=1, inplace=True)

In [ ]:
painel6 = painel6[painel6.V1014 == 6]

In [ ]:
painel6 = painel6[painel6.V20082 != 9999]

In [ ]:
painel6.to_csv('painel6.csv')

Filtrar, de acordo com o algoritmo proposto por [Ribas e Soares (2008)](https://www.ipea.gov.br/portal/index.php?option=com_content&view=article&id=4605), indivíduos (variável keyIden) que tenha data de nascimento e sexo iguais (variáveis V2007, V2008, V20081 e V20082)

In [ ]:
painel6 = pd.read_csv('painel6.csv')

In [ ]:
check_V20007 = painel6.groupby('keyInd').V2007.nunique().eq(1)
check_V20008 = painel6.groupby('keyInd').V2008.nunique().eq(1)
check_V200081 = painel6.groupby('keyInd').V20081.nunique().eq(1)
check_V200082 = painel6.groupby('keyInd').V20082.nunique().eq(1)

In [ ]:
keyInd = painel6.keyInd.unique()

In [ ]:
keyInd_V2007 = keyInd[check_V20007.values]
keyInd_V2008 = keyInd[check_V20008.values]
keyInd_V20081 = keyInd[check_V200081.values]
keyInd_V20082 = keyInd[check_V200082.values]

In [ ]:
intersect = np.intersect1d(keyInd_V2007, keyInd_V2008)
intersect = np.intersect1d(intersect, keyInd_V20081)
intersect = np.intersect1d(intersect, keyInd_V20082)

In [ ]:
mask = painel6.keyInd.isin(intersect)

In [ ]:
painel6_filtrado = painel6[mask]

In [ ]:
painel6_filtrado.to_csv('painel6_filtrado.csv', index=False)

In [2]:
painel6_filtrado = pd.read_csv('painel6_filtrado.csv', index_col=False)

Explorando os dados sobre declaração de cor ou raça

In [3]:
# número expressivo de respondentes mudaram a declaração de cor ou raça
mudou = painel6_filtrado[['keyInd', 'V2010']].groupby('keyInd').nunique().eq(1)

In [ ]:
lista_mudou = painel6_filtrado.keyInd.unique()[mudou.V2010.values]
mask_mudou = painel6_filtrado.keyInd.isin(lista_mudou)
painel6_mudou = painel6_filtrado[~mask_mudou]

True = não mudou; False = mudou a declaração de cor ou raça em ao menos uma das rodadas do painel

In [5]:
df_mudou = pd.DataFrame(mudou.value_counts())
df_mudou.columns = ['valor']
mudou_percent = df_mudou.values / painel6_filtrado.keyInd.nunique()
df_mudou['percent'] = mudou_percent
df_mudou

,valor,percent
V2010,,
True,350038,0.685894
False,160300,0.314106


Vamos criar lista com as observações da variável "cor ou raça" V2010 e filtrar apenas os indivíduos com 5 observações (participaram das 5 rodadas da PNAD Contínua painel 6)

In [6]:
def cria_lista(s):
    return s.to_list()
tipos_mudanca = painel6_mudou.groupby('keyInd').V2010.apply(cria_lista)

In [7]:
tipos_mudanca = tipos_mudanca[tipos_mudanca.map(len) == 5]

In [86]:
#mostrando os 50 primeiros maiores resultados que mostram a grande variedade de situações existentes
tipos_mudanca.value_counts()[:50]

TypeError: unhashable type: 'list'

Exception ignored in: 'pandas._libs.index.IndexEngine._call_map_locations'
Traceback (most recent call last):
  File "pandas/_libs/hashtable_class_helper.pxi", line 1709, in pandas._libs.hashtable.PyObjectHashTable.map_locations
TypeError: unhashable type: 'list'


[1, 4, 4, 4, 4]    6117
[4, 1, 4, 4, 4]    5128
[1, 1, 1, 1, 4]    4988
[4, 4, 4, 4, 1]    4758
[4, 1, 1, 1, 1]    4673
[4, 4, 1, 4, 4]    4535
[4, 4, 4, 1, 4]    4210
[1, 4, 1, 1, 1]    4009
[1, 1, 4, 1, 1]    3928
[1, 1, 1, 4, 1]    3844
[2, 4, 4, 4, 4]    3595
[4, 4, 4, 4, 2]    3150
[4, 2, 4, 4, 4]    2774
[1, 1, 4, 4, 4]    2710
[4, 4, 4, 2, 4]    2676
[4, 4, 2, 4, 4]    2601
[1, 1, 1, 4, 4]    2492
[4, 4, 1, 1, 1]    2140
[4, 4, 4, 1, 1]    2135
[4, 1, 1, 4, 4]    1654
[1, 4, 1, 4, 4]    1558
[1, 1, 4, 4, 1]    1514
[1, 4, 4, 4, 1]    1465
[1, 4, 4, 1, 1]    1462
[4, 4, 1, 1, 4]    1441
[1, 1, 4, 1, 4]    1432
[4, 1, 1, 1, 4]    1412
[4, 2, 2, 2, 2]    1350
[4, 4, 1, 4, 1]    1305
[1, 4, 4, 1, 4]    1296
[1, 4, 1, 1, 4]    1276
[4, 1, 4, 1, 1]    1250
[4, 1, 4, 1, 4]    1233
[4, 1, 4, 4, 1]    1210
[2, 2, 2, 2, 4]    1202
[2, 4, 2, 2, 2]    1173
[1, 4, 1, 4, 1]    1145
[4, 1, 1, 4, 1]    1135
[4, 4, 4, 2, 2]    1063
[2, 2, 4, 4, 4]    1054
[2, 2, 2, 4, 2]     991
[2, 2, 4, 2, 2] 

Percebemos uma grande diversidade de situações, mas a maioria das observações são combinações da cor branca (1), parda (4) e preta (2). Dessa maneira, consideraremos a primeira e a última observação (declaração inicial e final). Será notável um grande peso do grupo "outros", que se dá em parte pelas pessoas que, apesar de terem alterado sua declaração de raça durante o painel, declaram a mesma raça na primeira e última rodada.

In [9]:
def como_mudou(a):
    if a[0] == 1 and a[-1] == 4:
        return 1
    elif a[0] == 4 and a[-1] == 1:
        return 2
    elif a[0] == 2 and a[-1] == 4:
        return 3
    elif a[0] == 4 and a[-1] == 2:
        return 4
    elif a[0] == 1 and a[-1] == 2:
        return 5
    elif a[0] == 2 and a[-1] == 1:
        return 6
    else:
        return 7

legenda 1 = branco para pardo; 2 = pardo para branco; 3 = preto para pardo; 4 = pardo para preto; 5 = branco para preto; 6 = preto para branco; 7 = outros

In [10]:
t = tipos_mudanca.apply(como_mudou).value_counts()

In [11]:
#total de observações
tipos_mudanca.shape[0]

123360

In [12]:
pd.DataFrame({'contagem':t.values, 'percentagem':t.values / tipos_mudanca.shape[0]}, index=t.index)

,contagem,percentagem
7,60751,0.492469
1,22733,0.184282
2,19347,0.156834
3,9644,0.078178
4,9212,0.074676
5,860,0.006971
6,813,0.006590


Selecionares o grupo de pessoas que na primeira rodada (2017/4) se declaravam brancas e que estão em idade de inserção no ensino superior (estipulamos entre 18 e 22 anos). Considerando este grupo, a variável "ter mudado a declaração de cor para parda, preta ou indígena" (variável independente) tem associação com a variável "estar cursando curso no ensino superior" (variável dependente)?

Passo 1) Selecionar o grupo de pessoas que se declararam brancas em 2017/4 entre 18 e 22 anos

In [22]:
selecao = painel6_filtrado[(painel6_filtrado.Ano == 2017) & (painel6_filtrado.V2010 == 1) 
                           & (painel6_filtrado.V2009 >= 18) & (painel6_filtrado.V2009 <= 22)].keyInd

In [25]:
#total de observações encontradas
selecao.shape[0]

23041

In [28]:
painel6_filtrado.keyInd.unique().shape[0]

510338

In [26]:
df_selecao = painel6_filtrado[painel6_filtrado.keyInd.isin(selecao)]

In [27]:
df_selecao.shape

(53765, 32)

Passo 2) Criar as variáveis de interesse:

In [63]:
mudou_cor = df_selecao.groupby('keyInd')['V2010'].nunique().eq(1).map({True:1, False:0})
mudou_cor

keyInd
110000034.03.06.04.0     0
110000584.08.06.03.0     1
110000620.014.06.02.0    1
110000932.014.06.03.0    1
110001287.05.06.02.0     1
                        ..
530044198.07.06.02.0     1
530044434.01.06.05.0     1
530044434.012.06.06.0    1
530044434.03.06.03.0     1
530051067.09.06.02.0     1
Name: V2010, Length: 12634, dtype: int64

In [72]:
def ensino_superior(lista):
    for item in lista:
        if item >= 8:
            return 1
        return 0

In [74]:
ensino_superior = df_selecao.groupby('keyInd').V3003A.apply(cria_lista).apply(ensino_superior)

Passo 3) Criando tabela de contingência e aplicando o teste Qui Quadrado para testar se as variáveis são dependentes entre si

In [83]:
pd.crosstab(mudou_cor, ensino_superior, margins=True, margins_name='Total')

V3003A,0,1,Total
V2010,,,
0,3963,1139,5102
1,5034,2498,7532
Total,8997,3637,12634


In [76]:
crosstab = pd.crosstab(mudou_cor, ensino_superior)

In [77]:
stat, p, dof, expected = chi2_contingency(crosstab)

In [78]:
print(expected)

[[3633.26689884 1468.73310116]
 [5363.73310116 2168.26689884]]


In [79]:
prob = 0.995
critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
	print('Dependent (reject H0)')
else:
	print('Independent (fail to reject H0)')

probability=0.995, critical=7.879, stat=173.835
Dependent (reject H0)


A hipótese nula (de que não há associação ou dependência entre as variáveis "mudar a declaração de cor" e "acessar o ensino superior") foi rejeitada.